In [8]:
from preprocess_data import *
from config import config_base
import time
import utils
import pandas as pd
from data_pre import title_question
config = config_base.config

In [19]:
df = organize_data(config.train_data)

In [21]:
df = df[df['article_title'] == '《防务简报》2017年第8期']
df = df[df['question_type'] == '事实型问题']

In [22]:
df

,article_id,article_type,article_title,article_content,question_id,article_question,article_answer,question_type
53796,51293,防务快讯,《防务简报》2017年第8期,防务简报 2017 年第 8 期 28 所科技情报研究中心 总第 479 期 2017 年...,7529553f-ef6a-416b-bfd8-6ced5fb5de94,这篇新闻的来源？,《防务简报》,事实型问题


In [23]:
title = df['article_title'].values[0]
content = df['article_content'].values[0]
question = df['article_question'].values[0]

In [44]:
question

'这篇新闻的来源？'

In [ ]:
df[]

In [43]:
match(title, content, question, 500)

xx
aa


'  参考文献 [1] 梁海燕．以色列导弹防御系统部署现状及未来发展．飞航导弹，2011(12) [2] 以色列成功测试反导系统．http：／／news．xinhuanet．com／world／2009-04／07／content_11145228．html．新华网，2009-4-7 [3] 亚敏．以色列“铁穹”防御系统．http：／／news．163．com／12／1126／08／8H7MGC5S00014AED．html．网易新闻中心，2012-11-26 [4] 韩旭阳．以色列“铁穹”系统成功拦截加沙200余火箭弹．http：／／news．xinhuanet．com/world／2012-11／25／c_123998093．htm．中国新闻网，2012-11-25 [5] William Booth．Israel deploys Iron Dome defense system ahead of possible U．S．strike on Syria．http：／／articles．washingtonpost．com／2013-08-30／world／41608720_1-israeli-oeeupied-golan-heights-assad-s-allies-syria．The Washingtonpost．2013-8-30 [6] Rafael expands air and missile defense family of weapons http：／／defense-update．com／features／2008／june08／rafael_air_defense．htm．Defense Update．2008．8 [7] 钱钱，谢慧敏．美国以色列联合研制大卫投石索导弹防御系统．http：／／www．spacechina．com／n25／n144／n208／n232／c97740／content．html．中国航天科技集团网站，2008-8-1353  [8] Terminal missile defense interceptor．http：／／WWW．rafael．co．il／Marketing/186-1607-en／Marketing．aspx．Rafael Advance Defense Systems Ltd．2010 [9] 

In [7]:
min(contents_len)

90

In [10]:
df = pd.read_csv('data_gen/merge_500_val.csv')

In [26]:
import sys
from rouge import Rouge

In [27]:
sys.setrecursionlimit(1000000)
rouge = Rouge(metrics=['rouge-l'])

In [42]:
    def match(title, content, question, max_len):

        title_is_zh = utils.is_zh_or_en(title)
        content_is_zh = utils.is_zh_or_en(content)

        if title_is_zh:
            title_list = utils.split_word_zh(title)
        else:
            title = title.lower()
            title_list = utils.split_word_en(title)

        def count(flag, content_list):
            """ 查数 """
            number = 0
            for i in range(len(flag)):
                if flag[i] != 0:
                    number += len(content_list[i])+1
            return number

        # 过滤
        if content_is_zh:
            title_number = len(title_list)
            content_number = len(utils.split_word_zh(content))
            if (title_number + content_number + 1) <= max_len:
                return content

        if content_is_zh:
            if '。' not in content:
                c_list = utils.split_word_zh(content)
                c_list = c_list[: config.max_len-len(title_list)-1]
                return ''.join(c_list)

            content_list = content.split('。')
            temp = []
            for c in content_list:
                if c not in ['', ' ', '  ']:
                    temp.append(c)
            content_list = temp
            content_list = [utils.split_word_zh(c) for c in content_list]
            content_list = [title_list] + content_list
            content_len = len(content_list)

            if utils.is_zh_or_en(question):
                question_list = utils.split_word_zh(question)
            else:
                question = question.lower()
                question_list = utils.split_word_en(question)
            question_str = ' '.join(question_list)

            # 相似性得分: rouge-l
            scores = []
            for c in content_list:
                if ''.join(c) in question:
                    scores.append(-5)
                    continue
                c_str = ' '.join(c)
                score = rouge.get_scores(c_str, question_str, avg=True)['rouge-l']['r']
                scores.append(score)

            # 标记类型
            flag = np.zeros(content_len)
            # title_number = len(utils.split_word_zh(title))
            # max_len = max_len - title_number

            # 核心句:
            max_score = max(scores)
            for i in range(content_len):
                if scores[i] == max_score:
                    flag[i] = -1

            # 核心句下一句
            for i in range(content_len):
                if (flag[i] == -1) and (i+1 < content_len) and (flag[i+1] == 0):
                    flag[i+1] = -2

            # 最后一句
            if flag[-1] == 0:
                flag[-1] = -3

            # 第一句
            if flag[1] == 0:
                flag[1] = -4

            # 蕴含句（上+中+下）
            for i in range(content_len):
                if scores[i] == -5:
                    if (i-1 >= 0) and (flag[i-1] == 0):
                        flag[i-1] = -5
                    if flag[i] == 0:
                        flag[i] = -5
                    if (i+1 < content_len) and (flag[i+1] == 0):
                        flag[i+1] = -5

            # 核心句下下句
            for i in range(content_len):
                if (flag[i] == -1) and (i+2 < content_len) and (flag[i+2] == 0):
                    flag[i+2] = -6

            # 核心句上一句
            for i in range(content_len):
                if (flag[i] == -1) and (i-1 >= 0) and (flag[i-1] == 0):
                    flag[i-1] = -7

            # 核心句下下下句
            for i in range(content_len):
                if (flag[i] == -1) and (i+3 < content_len) and (flag[i+3] == 0):
                    flag[i+3] = -8

            # 核心句上上句
            for i in range(content_len):
                if (flag[i] == -1) and (i-2 >= 0) and (flag[i-2] == 0):
                    flag[i-2] = -9

            # 倒数第二句
            if(len(flag) >= 3) and (flag[-2] == 0):
                flag[-2] = -10

            # 第二句
            if (len(flag) >= 3) and (flag[2] == 0):
                flag[2] = -11

            flag[0] = 0
            number = count(flag, content_list)
            max_len = max_len - len(title_list)
            result = []
            if number <= max_len:
                for i in range(content_len):
                    if flag[i] != 0:
                        result.append(''.join(content_list[i]))
            else:
                print('xx')
                flag_copy = np.zeros(content_len)
                c_count = 0
                xxx = True
                for i in range(-1, -12, -1):
                    for j in range(len(flag)):
                        if flag[j] == i:
                            c_count = c_count + len(content_list[j]) + 1
                            if c_count <= max_len:
                                flag_copy[j] = -1
                            else:
                                xxx = False
                                break
                    if xxx is False:
                        break

                for i in range(content_len):
                    if flag_copy[i] == -1:
                        result.append(''.join(content_list[i]))
                if len(result) == 0:
                    print('aa')
                    for j in range(len(flag)):
                        if flag[j] == -1:
                            result = [''.join(content_list[j][: max_len-1])]
                            break

            # 过滤重复
            temp = []
            for r in result:
                if r not in temp:
                    temp.append(r)
            result = temp

            return '。'.join(result)

        else:
            www = content.lower()
            words = utils.split_word_en(www)
            if (len(words) + len(title_list) + 1) <= config.max_len:
                return content
            else:
                index = 0
                for i in words[: config.max_len-len(title_list)-1]:
                    index = index + len(i) + 1

                try:
                    while content[index-1] != ' ':
                        index -= 1
                except:
                    print(index)
                    print(content)

                return content[: index-1]